In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset_sim/mpc_motions_continuous.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(29, 30), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  243


In [6]:
motion.shape

(243, 31)

## Height-Force plots

In [7]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [8]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 6] < height)[0]
    fr_min_height = np.where(dataset[:, 9] < height)[0]
    rl_min_height = np.where(dataset[:, 12] < height)[0]
    rr_min_height = np.where(dataset[:, 15] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

### Footstep IDXs

In [9]:
footsteps_fwd = get_specific_cmd(motion, 1, 0, 0)
footsteps_clock = get_specific_cmd(motion, 0, 0, -1)
footsteps_count = get_specific_cmd(motion, 0, 0, 1)
footsteps_side_right = get_specific_cmd(motion, 0, -1, 0)
footsteps_side_left = get_specific_cmd(motion, 0, 1, 0)

In [10]:
print(footsteps_count.shape)

(0,)


# Dataset Preparation

In [11]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [12]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [13]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [18]:
def create_com_dataset(dataset, footsteps, motion=None, debug=False):
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 0:
            # Compute time difference between
            # current and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            #if time_difference < 1 and np.any(dataset[footsteps[idx-1], 1:4]) and np.any(dataset[footsteps[idx], 1:4]):
            if True:
                lf_rh_moving = contacts[footsteps[idx], 0]
                rf_lh_moving = contacts[footsteps[idx], 1]
                
                if lf_rh_moving == rf_lh_moving:
                    print("Invalid")
                    continue
                
                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 19:23])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 19:23])

                # Odom base vectors
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]

                # Rotate odometry estimation
                rotated_prev_base = prev_base @ R_prev
                rotated_curr_base = curr_base @ R_prev
                base_displacement = rotated_curr_base - rotated_prev_base
                
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 19:23])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 19:23])
                yaw = curr_yaw - prev_yaw
                
                # Handle angle wrap
                if np.sign(prev_yaw) != np.sign(curr_yaw):
                    print("Angle wrap detected.")
                    continue
                    
                inputs.append(dataset[footsteps[idx-1], 1:31].tolist() +
                              dataset[footsteps[idx], 1:4].tolist() +
                              [lf_rh_moving, rf_lh_moving])
                
                # Labels
                labels.append([base_displacement[0], base_displacement[1], yaw])
                
    return np.array(inputs, dtype=float), np.array(labels, dtype=float)
                
X_motion_fwd_CoM, Y_motion_fwd_CoM = create_com_dataset(motion, footsteps_fwd)
#X_motion_clock_CoM, Y_motion_clock_CoM = create_com_dataset(motion, footsteps_clock)
#X_motion_count_CoM, Y_motion_count_CoM = create_com_dataset(motion, footsteps_count)
#X_motion_side_left_CoM, Y_motion_side_left_CoM = create_com_dataset(motion, footsteps_side_left)
#X_motion_side_right_CoM, Y_motion_side_right_CoM = create_com_dataset(motion, footsteps_side_right)

# Stack datasets
#X_motion_CoM_continuous = np.concatenate((X_motion_fwd_CoM, X_motion_clock_CoM, X_motion_count_CoM))
#Y_motion_CoM_continuous = np.concatenate((Y_motion_fwd_CoM, Y_motion_clock_CoM, Y_motion_count_CoM))

X_motion_CoM_continuous = X_motion_fwd_CoM
Y_motion_CoM_continuous = Y_motion_fwd_CoM

print(X_motion_CoM_continuous.shape)
print(Y_motion_CoM_continuous.shape)

(242, 35)
(242, 3)


In [19]:
%store X_motion_CoM_continuous
%store Y_motion_CoM_continuous

Stored 'X_motion_CoM_continuous' (ndarray)
Stored 'Y_motion_CoM_continuous' (ndarray)


# Velocity Check

In [20]:
set(X_motion_CoM_continuous[:, 2])

{0.0}

# FWD CoM displacements

In [21]:
max(Y_motion_CoM_continuous[:, 0])

0.17179725628733422

In [22]:
# X displacement mean and std for each velocity
for velocity in np.arange(0.1, 1.1, 0.1):
    idx = np.where(np.isclose(X_motion_CoM_continuous[:, 0], velocity))[0]
    print(f"Velocity {velocity} with {len(idx)} samples")
    print(f"Mean CoM(x) displacement of {np.mean(Y_motion_CoM_continuous[idx, 0])} with a std of {np.std(Y_motion_CoM_continuous[idx, 0])}")
    print(f"Mean CoM(y) displacement of {np.mean(Y_motion_CoM_continuous[idx, 1])} with a std of {np.std(Y_motion_CoM_continuous[idx, 1])}")
    print(f"Mean CoM(yaw) displacement of {np.mean(Y_motion_CoM_continuous[idx, 2])} with a std of {np.std(Y_motion_CoM_continuous[idx, 2])}\n")

Velocity 0.1 with 34 samples
Mean CoM(x) displacement of 0.021480276092798997 with a std of 0.0014016037852279883
Mean CoM(y) displacement of -5.548113260294271e-05 with a std of 0.0004860003753254144
Mean CoM(yaw) displacement of -3.4540561016273856e-07 with a std of 0.0031023950172178075

Velocity 0.2 with 34 samples
Mean CoM(x) displacement of 0.04421817997597005 with a std of 0.004003139777983427
Mean CoM(y) displacement of 1.7263624928552737e-05 with a std of 0.0009693527555222108
Mean CoM(yaw) displacement of 0.00024619784080641497 with a std of 0.003098054498029205

Velocity 0.30000000000000004 with 31 samples
Mean CoM(x) displacement of 0.06589826514916622 with a std of 0.003805095839648057
Mean CoM(y) displacement of -4.262210631115097e-06 with a std of 0.0008222258891869572
Mean CoM(yaw) displacement of 9.340094646606657e-05 with a std of 0.0033219265065650245

Velocity 0.4 with 34 samples
Mean CoM(x) displacement of 0.08847336795703849 with a std of 0.0051392773186285405
Mea

/home/ilyass/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ilyass/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ilyass/.local/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/ilyass/.local/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/ilyass/.local/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# CK CoM displacement

In [ ]:
# X displacement mean and std for each velocity
for velocity in np.arange(0.1, 1.1, 0.1):
    idx = np.where(np.isclose(X_motion_CoM_continuous[:, 2], -velocity))[0]
    print(f"Velocity {velocity} with {len(idx)} samples")
    print(f"Mean CoM(x) displacement of {np.mean(Y_motion_CoM_continuous[idx, 0])} with a std of {np.std(Y_motion_CoM_continuous[idx, 0])}")
    print(f"Mean CoM(y) displacement of {np.mean(Y_motion_CoM_continuous[idx, 1])} with a std of {np.std(Y_motion_CoM_continuous[idx, 1])}")
    print(f"Mean CoM(yaw) displacement of {np.mean(Y_motion_CoM_continuous[idx, 2])} with a std of {np.std(Y_motion_CoM_continuous[idx, 2])}\n")

# CCK CoM displacement

In [ ]:
# X displacement mean and std for each velocity
for velocity in np.arange(0.1, 1.1, 0.1):
    idx = np.where(np.isclose(X_motion_CoM_continuous[:, 2], velocity))[0]
    print(f"Velocity {velocity} with {len(idx)} samples")
    print(f"Mean CoM(x) displacement of {np.mean(Y_motion_CoM_continuous[idx, 0])} with a std of {np.std(Y_motion_CoM_continuous[idx, 0])}")
    print(f"Mean CoM(y) displacement of {np.mean(Y_motion_CoM_continuous[idx, 1])} with a std of {np.std(Y_motion_CoM_continuous[idx, 1])}")
    print(f"Mean CoM(yaw) displacement of {np.mean(Y_motion_CoM_continuous[idx, 2])} with a std of {np.std(Y_motion_CoM_continuous[idx, 2])}\n")

### Fwd Plot

In [ ]:
np.arange(0.1, 0.9, 0.1)

In [ ]:
plt.plot(np.arange(0.1, 1.1, 0.1),
         [np.mean(Y_motion_fwd_CoM[np.where(np.isclose(X_motion_fwd_CoM[:, 0], velocity))[0], 0]) for velocity in np.arange(0.1, 1.1, 0.1)], 
         'ro')
plt.xlabel('Velocities')
plt.ylabel('\u0394X CoM')
plt.show()

In [ ]:
x = np.arange(0.1, 1.1, 0.1)
y = np.array([np.mean(Y_motion_fwd_CoM[np.where(np.isclose(X_motion_fwd_CoM[:, 0], velocity))[0], 0]) for velocity in np.arange(0.1, 1.1, 0.1)])
e = np.array([np.std(Y_motion_fwd_CoM[np.where(np.isclose(X_motion_fwd_CoM[:, 0], velocity))[0], 0]) for velocity in np.arange(0.1, 1.1, 0.1)])
plt.errorbar(x, y, e, linestyle='None', marker='s')
plt.xlabel('Velocities')
plt.ylabel('\u03B4X CoM')
plt.show()

### Left Plot

In [ ]:
plt.plot(np.arange(0.1, 1.1, 0.1),
         [np.mean(Y_motion_side_left_CoM[np.where(np.isclose(X_motion_side_left_CoM[:, 1], velocity))[0], 1]) for velocity in np.arange(0.1, 1.1, 0.1)], 
         'ro')
plt.xlabel('Velocities')
plt.ylabel('\u0394Y CoM')
plt.show()

In [ ]:
x = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
y = np.array([np.mean(Y_motion_side_left_CoM[np.where(np.isclose(X_motion_side_left_CoM[:, 1], velocity))[0], 1]) for velocity in np.arange(0.1, 1.0, 0.1)])
e = np.array([np.std(Y_motion_side_left_CoM[np.where(np.isclose(X_motion_side_left_CoM[:, 1], velocity))[0], 1]) for velocity in np.arange(0.1, 1.0, 0.1)])
plt.errorbar(x, y, e, linestyle='None', marker='s')
plt.xlabel('Velocities')
plt.ylabel('\u03B4Y CoM')
plt.show()

### Right Plot

In [ ]:
plt.plot([-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9],
         [np.mean(Y_motion_side_right_CoM[np.where(np.isclose(X_motion_side_right_CoM[:, 1], -velocity))[0], 1]) for velocity in np.arange(0.1, 1.0, 0.1)], 
         'ro')
plt.xlabel('Velocities')
plt.ylabel('\u0394Y CoM')
plt.show()

In [ ]:
x = np.array([-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9])
y = np.array([np.mean(Y_motion_side_right_CoM[np.where(np.isclose(X_motion_side_right_CoM[:, 1], -velocity))[0], 1]) for velocity in np.arange(0.1, 1.0, 0.1)])
e = np.array([np.std(Y_motion_side_right_CoM[np.where(np.isclose(X_motion_side_right_CoM[:, 1], -velocity))[0], 1]) for velocity in np.arange(0.1, 1.0, 0.1)])
plt.errorbar(x, y, e, linestyle='None', marker='s')
plt.xlabel('Velocities')
plt.ylabel('\u03B4Y CoM')
plt.show()

### Clock Plot

In [ ]:
plt.plot([-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1.0],
         [np.mean(Y_motion_clock_CoM[np.where(np.isclose(X_motion_clock_CoM[:, 2], -velocity))[0], 2]) for velocity in np.arange(0.1, 1.1, 0.1)], 
         'ro')
plt.xlabel('Velocities')
plt.ylabel('\u0394\u03B8 CoM')
plt.show()

In [ ]:
x = np.array([-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1.0])
y = np.array([np.mean(Y_motion_clock_CoM[np.where(np.isclose(X_motion_clock_CoM[:, 2], -velocity))[0], 2]) for velocity in np.arange(0.1, 1.1, 0.1)])
e = np.array([np.std(Y_motion_clock_CoM[np.where(np.isclose(X_motion_clock_CoM[:, 2], -velocity))[0], 2]) for velocity in np.arange(0.1, 1.1, 0.1)])
plt.errorbar(x, y, e, linestyle='None', marker='s')
plt.xlabel('Velocities')
plt.ylabel('\u03B4Y CoM')
plt.show()

### Count Plot

In [ ]:
plt.plot(np.arange(0.1, 1.1, 0.1),
         [np.mean(Y_motion_count_CoM[np.where(np.isclose(X_motion_count_CoM[:, 2], velocity))[0], 2]) for velocity in np.arange(0.1, 1.1, 0.1)], 
         'ro')
plt.xlabel('Velocities')
plt.ylabel('\u0394\u03B8 CoM')
plt.show()

In [ ]:
x = np.array(np.arange(0.1, 1.1, 0.1))
y = np.array([np.mean(Y_motion_count_CoM[np.where(np.isclose(X_motion_count_CoM[:, 2], velocity))[0], 2]) for velocity in np.arange(0.1, 1.1, 0.1)])
e = np.array([np.std(Y_motion_count_CoM[np.where(np.isclose(X_motion_count_CoM[:, 2], velocity))[0], 2]) for velocity in np.arange(0.1, 1.1, 0.1)])
plt.errorbar(x, y, e, linestyle='None', marker='s')
plt.xlabel('Velocities')
plt.ylabel('\u03B4Y CoM')
plt.show()

## Forwad Check

In [ ]:
print(f"Min displacement: {min(Y_motion_fwd_CoM[:, 0])}. Max displacement: {max(Y_motion_fwd_CoM[:, 0])}")

## Clockwise Check

In [ ]:
print(f"Min displacement: {min(Y_motion_clock_CoM[:, 2])}. Max displacement: {max(Y_motion_clock_CoM[:, 2])}")

## Counter Check

In [ ]:
print(f"Min displacement: {min(Y_motion_count_CoM[:, 2])}. Max displacement: {max(Y_motion_count_CoM[:, 2])}")

## Left Check

In [ ]:
print(f"Min displacement: {min(Y_motion_side_left_CoM[:, 1])}. Max displacement: {max(Y_motion_side_left_CoM[:, 1])}")

## Right Check

In [ ]:
print(f"Min displacement: {min(Y_motion_side_right_CoM[:, 1])}. Max displacement: {max(Y_motion_side_right_CoM[:, 1])}")